In [1]:
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import plotly.offline as py


In [2]:
spark = SparkSession.builder.appName("flights").getOrCreate()
import json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType
with open("../../util/schema.json","r") as f:
    schema = StructType.fromJson(json.load(f))

23/01/03 15:21:13 WARN Utils: Your hostname, MacBook-Air-di-Teodoro.local resolves to a loopback address: 127.0.0.1; using 192.168.135.184 instead (on interface en0)
23/01/03 15:21:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/03 15:21:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/03 15:21:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.read.csv("../../data.nosync/cleaned/cleaned_flights.csv",schema=schema, header=True)

In [4]:
# create a new column 'label' that is 1 if the flight is delayed and 0 if it is not
df = df.withColumn("label", when(df["ArrDelay"] > 0, 1).otherwise(0))

In [5]:
df.columns

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'Tail_Number',
 'Flight_Number_Reporting_Airline',
 'Origin',
 'Dest',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Distance',
 'DistanceGroup',
 'DivAirportLandings',
 'ORIGIN_STATE',
 'ORIGIN_LATITUDE',
 'ORIGIN_LONGITUDE',
 'DEST_STATE',
 'DEST_LATITUDE',
 'DEST_LONGITUDE',
 'label']

In [6]:
features = [
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'Reporting_Airline',
 'Origin',
 'Dest',
 #'DepDelay',
 'CRSDepTime',
 'CRSArrTime',
 'CRSElapsedTime',
 'AirTime',
 'Distance',
 'ORIGIN_STATE',
 'DEST_STATE',
 'label'
 ]


# mantain only the features in features list
df = df.select(features)

In [7]:
df = df.sample(False,0.07,seed=42)

In [8]:
# get the number of rows in the dataframe
df.count()

437379

In [ ]:
df.show(10)

In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.feature import OneHotEncoder

In [10]:
df.dtypes

[('Quarter', 'int'),
 ('Month', 'int'),
 ('DayofMonth', 'int'),
 ('DayOfWeek', 'int'),
 ('Reporting_Airline', 'string'),
 ('Origin', 'string'),
 ('Dest', 'string'),
 ('CRSDepTime', 'int'),
 ('CRSArrTime', 'int'),
 ('CRSElapsedTime', 'double'),
 ('AirTime', 'double'),
 ('Distance', 'double'),
 ('ORIGIN_STATE', 'string'),
 ('DEST_STATE', 'string'),
 ('label', 'int')]

In [11]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

categoricalCols = [field for (field, dataType) in df.dtypes
if dataType == "string"]

indexOutputCols = [x + "Index" for x in categoricalCols]
oheOutputCols = [x + "OHE" for x in categoricalCols]

stringIndexer = StringIndexer(inputCols=categoricalCols,
                                outputCols=indexOutputCols,
                                handleInvalid="skip")
oheEncoder = OneHotEncoder(inputCols=indexOutputCols,
                            outputCols=oheOutputCols)

numericCols = [field for (field, dataType) in df.dtypes
                    if ((dataType == "double" or dataType == "int" ) & (field != "label"))]

assemblerInputs = oheOutputCols + numericCols

vecAssembler = VectorAssembler(inputCols=assemblerInputs,
                        outputCol="features")


In [12]:
# create the pipeline
pipeline = Pipeline(stages=[stringIndexer, oheEncoder, vecAssembler])

# fit the pipeline to the data
pipelineModel = pipeline.fit(df)

# transform the data
df_proc = pipelineModel.transform(df)

In [13]:
# select the features and label columns
df_proc = df_proc.select("features","label")

In [14]:
# show the first 5 rows
df_proc.show(5)


23/01/03 14:58:15 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------------------+-----+
|            features|label|
+--------------------+-----+
|(758,[8,98,328,66...|    1|
|(758,[8,98,328,66...|    0|
|(758,[8,25,375,65...|    1|
|(758,[8,51,338,64...|    0|
|(758,[8,51,338,64...|    1|
+--------------------+-----+
only showing top 5 rows



In [ ]:
# count the number of rows with label 1
df_proc.filter(df_proc.label == 1).count()

In [ ]:
# count the number of rows with label 0
df_proc.filter(df_proc.label == 0).count()

In [15]:
sample_rate = 2501251/3745488
sample_rate

0.6678037681605175

In [16]:
# split the data into train and test
train, test = df_proc.randomSplit([0.8, 0.2], seed=42)


In [17]:

train_1 = train.filter(train.label == 1)


train_0 = train.filter(train.label == 0).sample(False, sample_rate, seed=42)


# merge the two datasets
train = train_1.union(train_0)

In [18]:
train = train.orderBy(rand())

In [19]:
# count the number of rows with label 1
train.filter(train.label == 1).count()

140519

In [20]:
# count the number of rows with label 0
train.filter(train.label == 0).count()

140354

In [ ]:
# use a grid search to find the best hyperparameters
rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed=42)
paramGrid = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [6,15]) \
    .addGrid(rf.maxBins, [60,100]) \
    .addGrid(rf.numTrees, [60, 120]) \
    .build()

# use the grid with 8-fold cross validation
crossval = CrossValidator(estimator=rf,
                            estimatorParamMaps=paramGrid,
                            evaluator=MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy"),
                            numFolds=5)

# fit the model
cvModel = crossval.fit(train)

# get the best model
bestModel = cvModel.bestModel

# get the best hyperparameters
bestModel.extractParamMap()


In [ ]:
# create the model
rf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=30,maxDepth=10)


In [ ]:
# train the model
model = rf.fit(train)

In [ ]:
# evaluate the model
predictions = model.transform(train)

# evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

# print the accuracy
print("Accuracy = %g " % accuracy)



Test Error = 0.419311 
Accuracy = 0.580689 

In [ ]:
# show the predictions and the true label
predictions.select("prediction", "label").show(10)